https://www.uscis.gov/tools/reports-studies/immigration-forms-data/data-set-form-i-485-application-adjustment-status

In [1]:
import os, time, re
import collections
import pandas as pd
import numpy as np

In [2]:
input_dir = os.path.join(os.getcwd(),'input')

In [3]:
os.path.exists(input_dir) or os.mkdir(input_dir)

True

In [4]:
pd.set_option('display.max_columns', 500)

In [5]:
ex = pd.read_csv(os.path.join(input_dir,'fy2018_qtr1.csv'))

In [6]:
# drop columns between location (first 3 columns) and total numbers (last four colums)
ex1 = ex.iloc[:,0:3].join(ex.iloc[:,-4:])

In [7]:
def clean_up(df):
    """
    Sets clean columns and deletes unwanted rows
    """
    #set dict of clean columns
    clean_columns = collections.OrderedDict((
        (df.columns[0], 'state'),
        (df.columns[1], 'city'),
        (df.columns[2], 'office_code'), 
        (df.columns[3], 'app_received'),
        (df.columns[4], 'app_approved'),
        (df.columns[5], 'app_denied'),
        (df.columns[6], 'app_pending'),
    ))
    
    #apply it to columns 
    df.rename(columns=clean_columns,inplace=True)
    
    #drop first rows that we are not using 
    df.drop([0], inplace=True)
    
    #drop rows that repeat the lable (first of second and third pages)
    df.dropna(subset=['state','city','office_code'], how='all', inplace=True)
    
    #fill out the state column
    df.loc[:,'state'].fillna(method='ffill', inplace=True)
    
    #strip white spaces 
    df = df.apply(lambda x: x.str.strip())
    
    #fill out all the empty strings
    df.replace('', np.nan, inplace=True)
    
    #drop where last four columns are null
    df.dropna(subset=['app_received','app_approved','app_denied','app_pending'], how='all', inplace=True)
    
    
    return df
    

In [8]:
ex2 = clean_up(ex1)

In [9]:
ex2

,state,city,office_code,app_received,app_approved,app_denied,app_pending
4,Total,NaN,NaN,"180,562","142,782","12,052","650,479"
8,Alabama,Montgomery,MGA,664,225,31,"2,950"
10,Alaska,Anchorage,ANC,111,99,17,319
12,Arizona,Phoenix,PHO,"1,447",888,131,"5,866"
13,Arizona,Tucson,TUC,409,731,33,"1,250"
15,Arkansas,Fort Smith,FSA,169,345,25,661
17,California,Chula Vista,CVC,-,-,-,-
18,California,Fresno,FRE,902,814,88,"3,090"
19,California,Imperial,IMP,83,123,D,249
20,California,Los Angeles,LOS,"2,318","2,022",167,"7,677"
